<a href="https://colab.research.google.com/github/prinaldi3/Denoising/blob/main/SG_vs_CNN_Exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Installations

In [ ]:
!python -m pip install pip==21.0.1

In [ ]:
!rm -rf xca
!git clone https://github.com/maffettone/xca

In [ ]:
%%bash
cd xca
python -m pip install .

In [ ]:
import xca
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from xca.ml.tf_models import build_CNN_encoder_model, build_CNN_decoder_model, VAE_denoising_training, VAE

Preprocessing Dataset

In [ ]:
from xca.ml.tf_data_proc import build_dataset

#these control the degree/intensity of noise
log_noise_min= -1 
log_noise_max= -.5

# Build dataset
def preprocess(data, label):
    X = tf.cast(data, tf.float32)
    X = (X - tf.math.reduce_min(X, axis=0, keepdims=True)) / (
        tf.math.reduce_max(X, axis=0, keepdims=True)
        - tf.math.reduce_min(X, axis=0, keepdims=True)
    )
    noisy = tf.cast(data, tf.float32) + tf.random.normal(
        data.shape,
        stddev=10 ** np.random.uniform(log_noise_min, log_noise_max),
        dtype=tf.float32,
    )
    noisy = (noisy - tf.math.reduce_min(noisy, axis=0, keepdims=True)) / (
        tf.math.reduce_max(noisy, axis=0, keepdims=True)
        - tf.math.reduce_min(noisy, axis=0, keepdims=True)
    )
    return {"X": X, "X_noisy": noisy, "label": label}


In [ ]:
#change as needed
dataset_paths = ['/content/gdrive/MyDrive/tuning_plots_data_9/tfrecords/data.tfrecord']

In [ ]:
#Now preprocess guassian data
batch_size = 8
data,_ = build_dataset(
        dataset_paths=dataset_paths,
        batch_size=batch_size,
        multiprocessing=1,
        categorical=True,
        val_split=0.0,
        data_shape=(500,1),
        preprocess=preprocess,
    )

Load Model and Plot Comparison 

In [ ]:
#Define and load model
encoder, last_conv_shape = build_CNN_encoder_model(data_shape=(500,1), latent_dim=100, dense_dims=[], filters=[32, 32, 16], kernel_sizes=[16, 16, 16],  strides=[1, 1, 1], pool_sizes=[1, 1, 1], paddings=["same"]*3, verbose=True)

decoder = build_CNN_decoder_model(data_shape=(500,1), latent_dim=100, last_conv_layer_shape=last_conv_shape, filters = [16, 32, 32], kernel_sizes=[16, 16, 16], strides=[1, 1, 1], paddings=["same"]*4, verbose=True)
vae = VAE(encoder, decoder, 0.)

cnn_checkpoint_path = '/content/gdrive/MyDrive/guassian_denoise_95/training_checkpoints'
checkpoint = tf.train.Checkpoint(model=vae)
checkpoint.restore(tf.train.latest_checkpoint(cnn_checkpoint_path))

In [ ]:
from scipy.signal import savgol_filter

#Plot a comparison between CNN reconstruction and Savitzky-Golay smoothing
for batch in data:
  #remove noise from data with CNN
  output = vae(batch["X_noisy"], training=False)
  X_denoise_CNN = output["reconstruction"][0]

  X = batch["X"][0]
  X_noisy = batch["X_noisy"][0]

  #smooth out data with Savitzky-Golay filter
  window_size = 45
  poly_degree = 10
  deriv = 0
  X_noisy = np.reshape(X_noisy, (500,))
  X_denoise_SG = savgol_filter(X_noisy, window_size, poly_degree, deriv=deriv) #fit to polynomial of degree 10

  label = batch["label"][0]
  fig, ax = plt.subplots(figsize=(10, 10))

  #Plot reconstructions (as well as raw, true data) on top of one another
  ax.plot(X_noisy + 3, label="Raw", color='gray')
  ax.plot(X + 2, label="True", color='k')
  ax.plot(X_denoise_CNN + 1, 'r', label="CNN Denoise")
  ax.plot(X_denoise_SG, 'b', label="SG Denoise")

  ax.set_title("Comparison of CNN and Savitzky-Golay Denoising (Guassian data)")
  ax.set_xlabel("Pixel #")
  ax.set_ylabel("Intensity [Arb.]")

  ax.get_yaxis().set_ticks([])
  #fig.savefig('/content/gdrive/MyDrive/poster_figs/CNN_SG_comparison_guassian_window45', facecolor='white')
  plt.legend()
  break